# HW05: Word Embeddings

Remember that these homework work as a completion grade. **You can <span style="color:red">not</span> skip one section this homework.**

**Essay Feedback**

Please provide feedback to two classmates' essays on Eduflow.

**Training word2vec**

In this section, we train a word2vec model using gensim. We train the model on text8 (which consists of the first 90M characters of a Wikipedia dump from 2006 and is considered one of the benchmarks for evaluating language models).

In [1]:
import gensim.downloader as api

api.info("text8")

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [2]:
dataset = api.load("text8")

[==================================================] 100.0% 31.6/31.6MB downloaded


In [5]:
from gensim.models import Word2Vec

##TODO train a word2vec model on this dataset which appear at least 10 times in the corpus

model = Word2Vec(dataset, min_count=10)

**Word Similarities**

gensim models provide almost all the utility you might want to wish for to perform standard word similarity tasks. They are available in the .wv (wordvectors) attribute of the model, more details could be found [here](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [8]:
model.wv

##TODO find the closest words to king

close_words = model.wv.most_similar('king')

for word in close_words:
    print(word)

('prince', 0.7743529081344604)
('queen', 0.7292153239250183)
('kings', 0.7031580209732056)
('throne', 0.6981783509254456)
('emperor', 0.6948561072349548)
('regent', 0.6773486137390137)
('sultan', 0.6719624400138855)
('vii', 0.6605079174041748)
('aragon', 0.6547995209693909)
('elector', 0.6542447209358215)


King is to man as woman is to X

In [9]:
##TODO find the closest word for the vector "woman" + "king" - "man"

closest_word = model.wv.most_similar(positive=['woman', 'king'], negative=['man'])[0]

print(closest_word)

('queen', 0.6954731941223145)


**Evaluate Word Similarities** 

One common way to evaluate word2vec models are word analogy tasks. Let's check how good our model is on one of those. We consider the [WordSim353](http://alfonseca.org/eng/research/wordsim353.html) benchmark, the task is to determine how similar two words are.

In [10]:
!wget http://alfonseca.org/pubs/ws353simrel.tar.gz
!tar xf ws353simrel.tar.gz

path = "wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"

def load_data(path):
    X, y = [], []
    with open(path) as f:
        for line in f:
            line = line.strip().split("\t")
            X.append((line[0], line[1])) # each entry in x contains two words, e.g. X[0] = (tiger, cat)
            y.append(float(line[-1])) # each entry in y is the annotation how similar two words are, e.g. Y[0] = 7.35
    return X, y

X, y = load_data(path)
print (X[:3], y[:3])

--2023-03-30 13:34:53--  http://alfonseca.org/pubs/ws353simrel.tar.gz
Resolving alfonseca.org (alfonseca.org)... 162.215.249.67
Connecting to alfonseca.org (alfonseca.org)|162.215.249.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5460 (5.3K) [application/x-gzip]
Saving to: ‘ws353simrel.tar.gz’

ws353simrel.tar.gz  100%[===================>]   5.33K  --.-KB/s    in 0s      

2023-03-30 13:34:54 (486 MB/s) - ‘ws353simrel.tar.gz’ saved [5460/5460]

[('tiger', 'cat'), ('tiger', 'tiger'), ('plane', 'car')] [7.35, 10.0, 5.77]


In [14]:
##TODO compute how similar the pairs in the WordSim353 are according to our model
# if a word is not present in our model, we assign similarity 0 for the respective text pair

sim_scores = []

for pair in X:
    try:
        score = model.wv.similarity(pair[0], pair[1])
    except KeyError:
        score = 0.0
    
    sim_scores.append(score)

print(sim_scores[:3])

[0.61210567, 1.0, 0.4190455]


In [17]:
from scipy.stats import spearmanr

##TODO compute spearman's rank correlation between our prediction and the human annotations

correlation, _ = spearmanr(y, sim_scores)

print(correlation)

0.6463631577377009


In [19]:
import spacy
en = spacy.load('en_core_web_sm')

##TODO compute word similarities in the WordSim353 dataset using spaCy word embeddings

sim_scores = []

for pair in X:
    word1 = en(pair[0])
    word2 = en(pair[1])

    score = word1.similarity(word2)
    sim_scores.append(score)

print(sim_scores[:3])

##TODO compute spearman's rank correlation between these similarities and the human annotations
# Don't worry if results are not too convincing for this experiment

correlation, _ = spearmanr(y, sim_scores)

print(correlation)

<ipython-input-19-e68eac66da0e>:12: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  score = word1.similarity(word2)


[0.6628428894742415, 1.0, 0.7974166052761725]
0.0917488312498204


**PyTorch Embeddings**

In [52]:
#Import the AG news dataset (same as hw01)
#Download them from here 
# !wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]

df = df.sample(n=10000) # # only use 10K datapoints

label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]

y = df['label']

df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]

df.head()

,label,title,lead,text
15599,world,12 Nepalese Reportedly Executed in Iraq,A video purporting to show the killings of Nep...,12 Nepalese Reportedly Executed in Iraq A vide...
17422,sport,NL notables,The Mets' Jeff Keppinger got his first major l...,NL notables The Mets' Jeff Keppinger got his f...
82962,world,Kidnappers Extend Deadline on British Hostage,Militants who claim to be holding Northern Iri...,Kidnappers Extend Deadline on British Hostage ...
66124,sci/tech,Intel to Ship Dual-Core Xeon in 2006,"Company denies the chip has been delayed, but ...",Intel to Ship Dual-Core Xeon in 2006 Company d...
56694,sport,Bottom line: Boston believes,These Boston Red Sox aren #39;t worried about ...,Bottom line: Boston believes These Boston Red ...


In [46]:
vocab = 200
##TODO tokenize the text, only keep 200 most frequent words 

from gensim.utils import tokenize
from collections import Counter

tokenized_text = [list(tokenize(t, lowercase = True)) for t in df['text']]


whole_corpus = []

for t in tokenized_text:
    whole_corpus.extend(t)


common_words = set([t[0] for t in Counter(whole_corpus).most_common(200)])

print(common_words)

{'more', 'world', 'to', 'he', 'president', 'former', 'first', 'court', 'space', 'n', 'as', 'microsoft', 'gt', 'no', 'and', 'would', 'years', 'b', 'into', 'a', 'u', 'had', 'you', 'international', 'american', 'get', 'next', 'reported', 'what', 'oil', 'bush', 'this', 'google', 'their', 'co', 'prime', 'national', 'day', 'japan', 'india', 's', 'make', 'afp', 'officials', 'the', 'reuters', 'yesterday', 'been', 'third', 'year', 'after', 'week', 'software', 'company', 'they', 'it', 'york', 'deal', 'victory', 'quarter', 'just', 'computer', 'market', 'killed', 'may', 'expected', 'news', 'of', 'billion', 'said', 'night', 'washington', 'wednesday', 'could', 'announced', 'profit', 'high', 'when', 'end', 'will', 'home', 'by', 'says', 'plans', 'against', 'about', 't', 'last', 'lead', 'season', 'four', 'out', 'tuesday', 'be', 'three', 'baghdad', 'its', 'an', 'thursday', 'million', 'chief', 'but', 'cup', 'service', 'some', 'one', 'second', 'report', 'won', 'two', 'open', 'games', 'china', 'red', 'most'

In [47]:
length = 100
#TODO create a one_hot representation for each word and truncate/pad the sequences such that they are all of the same length (here we use 100)

# !pip install Keras-Preprocessing
from keras.preprocessing.text import one_hot
from keras_preprocessing.sequence import pad_sequences

X_one_hot = [one_hot(opinion, n=10000) for opinion in df['text']]
print(X_one_hot[0][:50])

X_one_hot_padded = pad_sequences(X_one_hot, padding='post', maxlen=100, truncating='post')
X_one_hot_padded.shape

[3054, 4162, 469, 4976, 1534, 294, 5894, 5741, 8172, 272, 8172, 5671, 1474, 9534, 5184, 5612, 3054, 4487, 1323, 3181, 6073, 4237, 2466, 1323, 3181, 4736, 3715, 227, 2464, 3181, 1736, 612, 1534, 294, 7741, 5894, 5741, 469, 7903, 1534, 8172]


(10000, 100)

In [66]:
##TODO create your torch embedding like we did in notebook 5! (hint: predicting labels: world, sport, business, and sci/tech)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import math
import numpy as np

class EmbeddingNet(nn.Module):
    def __init__(self, num_words=100):
        super(EmbeddingNet, self).__init__()
        self.embedding = nn.Embedding(num_words + 1, 96)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(96 * 13924, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 4)

    def forward(self, x):
        x = self.embedding(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x, _ = torch.max(x, dim=-1)
        return x

class GenericDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tsize = math.ceil(0.1 * len(y))

X_train, X_valid, y_train, y_valid = train_test_split(X_one_hot_padded, np.array(y), test_size=tsize, stratify=y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=tsize, stratify=y_train, random_state=0)

train_dataset = GenericDataset(X_train, y_train)
valid_dataset = GenericDataset(X_valid, y_valid)
test_dataset = GenericDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True, drop_last=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=16, shuffle=False, drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False, drop_last=True)

model = EmbeddingNet()

In [67]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [68]:
from sklearn.metrics import accuracy_score, f1_score

num_epochs = 20

f1_scores = []
best_model = None
train_stop = 5

for epoch in range(num_epochs):
    
    model.train()
    
    train_loss = []
    valid_loss = []

    for i, (inputs, labels) in enumerate(train_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Expected input datatype is float
        # Also guarantees higher precision than int
        outputs = model(inputs.int())
        loss = criterion(outputs.int(), labels.int().unsqueeze(1))
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    model.eval()

    pred_labels = []
    true_labels = []

    with torch.no_grad():
        for j, (inputs, labels) in enumerate(valid_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs.int())
            loss = criterion(outputs.int(), labels.int().unsqueeze(1))

            valid_loss.append(loss.item())

            predicted = [1 if d > 0.5 else 0 for d in outputs.data.squeeze()]

            pred_labels.extend(predicted)
            true_labels.extend(list(labels.detach().cpu().numpy()))

    accuracy = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)

    f1_scores.append(f1)

    if f1 >= max(f1_scores):
        best_model = model.state_dict()
    
    print('epoch {}: train loss: {:.4f}, valid loss: {:.4f}, acc: {:.4f}, f1: {:.4f}'.format(epoch+1, np.mean(train_loss), np.mean(valid_loss), accuracy, f1))

    if len(f1_scores) - f1_scores.index(max(f1_scores)) == train_stop:
        break

torch.save(best_model, 'best_model.pt')


model.load_state_dict(torch.load('best_model.pt'))

model.eval()

pred_labels = []
true_labels = []

with torch.no_grad():
    for j, (inputs, labels) in enumerate(test_loader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs.int())

        predicted = [1 if d > 0.5 else 0 for d in outputs.data.squeeze()]

        pred_labels.extend(predicted)
        true_labels.extend(list(labels.detach().cpu().numpy()))

    accuracy = accuracy_score(true_labels, pred_labels)

print()
print('Best model reaches accuracy of {:.2f}%'.format(accuracy*100))

tensor([[5889, 7719,  870,  ...,    0,    0,    0],
        [1296, 8160, 2478,  ...,    0,    0,    0],
        [ 199,  699, 8831,  ...,    0,    0,    0],
        ...,
        [9184, 3304, 6987,  ...,    0,    0,    0],
        [8430, 9268, 2582,  ...,    0,    0,    0],
        [7719, 3138, 6555,  ...,    0,    0,    0]], dtype=torch.int32)


IndexError: ignored